In [1]:
import cvxpy as cp
import numpy as np
from scipy.linalg import sqrtm
import random as r
import pandas as pd
from qpth.qp import QPFunction
import torch
from sklearn.datasets import make_spd_matrix

In [2]:
global n
n=6       #length of the optimiation variable
global m
m=6       #no.of inequality constraints
global p
p=3       #no.of equality constraints

In [3]:
def find_valid(seeds):
    #find seeds for which the corresponding randomly generated problem yields a solution (from cvxpy)
    seeds_old=seeds.copy()
    seeds_old_old=seeds_old.copy()
    while True:        
        while True: 
            for k,s in enumerate(seeds):
                r.seed(s)
                P=make_spd_matrix(n, random_state=s)
                P = sqrtm(P)
                q=np.array([r.randint(0,9) for i in range(n)]).reshape(n).astype(float)
                G=np.array([r.randint(0,9)*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
                h=np.array([0 for i in range(m)]).reshape(n).astype(float)
                A=np.array([r.randint(0,9) for i in range(p*n)]).reshape(p,n).astype(float)
                b=np.array([r.randint(0,9) for i in range(p)]).reshape(p).astype(float)
                try:
                    x,y=qp_cvxpy(n,P,q,G,h,A,b)
                except:
                    #if solving the problem creates error, remove this seed from list
                    seeds.remove(s)
                    continue
                #if the problem is infeasible, y is None; if so remove this seed from list
                if(np.array(y).any()==None):
                    seeds.remove(s)
            if len(seeds_old)==len(seeds):
                break
            else:
                seeds_old=seeds
        if len(seeds_old_old)==len(seeds_old):
            break
        seeds_old_old=seeds_old
    print("No.of problems generated: ",len(seeds)) 
    return(seeds)

In [4]:
def qp_cvxpy(n,P,q,G,h,A,b):
    x= cp.Variable(n)
    objective=cp.Minimize((1/2)*cp.sum_squares(P*x) + q.T @ x)
    constraints=[G@x <=h,A@x ==b]
    prob=cp.Problem(objective,constraints)
    prob.solve(verbose=False)
    cvxpy_optimum= prob.value
    cvxpy_solution=x.value
    return cvxpy_optimum,cvxpy_solution

In [5]:
#initialize seeds for generating random quadratic problems
try_seeds=[i for i in range(100,10000,10)]
# print(len(try_seeds))
seeds= find_valid(try_seeds)

No.of problems generated:  639


In [ ]:
columns=["seed"]+[str(i)+"cp" for i in range(n)]
results=pd.DataFrame([],columns=columns)
for k,s in enumerate(seeds):
    r.seed(s)
    P=make_spd_matrix(n, random_state=s)
    q=np.array([r.randint(0,9) for i in range(n)]).reshape(n).astype(float)
    G=np.array([r.randint(0,9)*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
    h=np.array([0 for i in range(m)]).reshape(n).astype(float)
    A=np.array([r.randint(0,9) for i in range(p*n)]).reshape(p,n).astype(float)
    b=np.array([r.randint(0,9) for i in range(p)]).reshape(p).astype(float)
    
    P_tch=torch.from_numpy(P)
    q_tch=torch.from_numpy(q)
    G_tch=torch.from_numpy(G)
    h_tch=torch.from_numpy(h)
    A_tch=torch.from_numpy(A)
    b_tch=torch.from_numpy(b)
    
    
    P = sqrtm(P)
    x,y=qp_cvxpy(n,P,q,G,h,A,b)
    re=pd.Series({"seed":s})
    for i in range(n):
        re=re.append(pd.Series({str(i)+"cp":np.round(y[i],6)}))
    
    qpf = QPFunction()
    try:
        solution = qpf(P_tch, q_tch, G_tch, h_tch, A_tch, b_tch)
        qpth_solution=solution.detach().numpy()[0]
    except:
        qpth_solution=[0]*n
    for i in range(n):
        re=re.append(pd.Series({str(i)+"qpt":np.round(qpth_solution[i],6)}))
    results=results.append(re,ignore_index=True )
    


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------



In [ ]:
error=np.zeros(len(results))
for i in range(n):
    error+=results[str(i)+"cp"]-results[str(i)+"qpt"]
results["cp-qpt-error"]=error    
results["cp-qpt-er_flag"]=error!=0

print("No of solved problems",len(seeds))
print("No of inaccurate results",len(error[error!=0]))

results[results["cp-qpt-er_flag"]]

In [ ]:
results.to_csv("test_cases.csv")